In [1]:
import pandas as pd
import requests
import json
import time
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from sodapy import Socrata
from cdp_secrets import token
print('completed at ' + str(datetime.now()))

completed at 2023-10-10 23:17:51.250644


In [27]:
#Constants
chicago_url="data.cityofchicago.org"
crime_root="ijzp-q8t2"
district_root="z8bn-74gv"
beat_root="n9it-hstw"
ca_root="igwz-8jzy"

#Chicago Data Portal Connection
cdp = Socrata(chicago_url,
                 token
             )
print('completed at ' + str(datetime.now()))

completed at 2023-10-10 23:29:53.456664


In [5]:
def callAPI(root: str, filter: str) -> pd.DataFrame:
    results=cdp.get(root, query=filter)
    df=pd.DataFrame.from_records(results)
    return df
#print('completed at ' + str(datetime.now())

In [6]:
# Get dates for the last 18 months
today = datetime.now().strftime("%Y-%m-%d")
eighteen_months = (datetime.now() - relativedelta(months=18)).strftime("%Y-%m-%d")

print(today)
print(eighteen_months)
print('completed at ' + str(datetime.now()))

2023-10-10
2022-04-10
completed at 2023-10-10 23:18:21.825634


In [31]:
#limit required for SoQL query, as we are filtering for one year limit is set high to capture all records
limit='1000000000'

#Filters use SQL structure, some where clauses use a different format. Refer to API Documentation for correct syntax

crime_filter="""Select distinct beat, ward, community_area, district limit %s"""% (limit)

district_filter="""Select distinct district,district_name limit 1000"""

beat_filter="""Select beat_num,district limit 300"""

ca_filter="""Select area_numbe, community"""

print('completed at ' + str(datetime.now()))

completed at 2023-10-10 23:32:15.321794


In [24]:
#crime_table
crime_df=callAPI(crime_root,crime_filter)

print('completed at ' + str(datetime.now()))

completed at 2023-10-10 23:22:49.397630


In [32]:
ca_df=callAPI(ca_root,ca_filter)

In [8]:
#Beat To District Key Table
#To be Replaced with neighborhood table

#Police Beat
beat_df=callAPI(beat_root,beat_filter)

#Remove leading 0 from district field
beat_df['district']=pd.to_numeric(beat_df['district'])
beat_df['district']=beat_df['district'].apply(lambda x: str(x))

#Remove leading 0 from beat_num field
beat_df['beat_num']=pd.to_numeric(beat_df['beat_num'])
beat_df['beat_num']=beat_df['beat_num'].apply(lambda x: str(x))

#District table
district_df=callAPI(district_root,district_filter)

#join together to create matching table
district_key=pd.merge(beat_df,district_df,on='district',how='left')

print(district_key)
print('completed at ' + str(datetime.now()))

    beat_num district   district_name
0       1713       17     Albany Park
1       3100       31             NaN
2       1651       16  Jefferson Park
3       1914       19       Town Hall
4       1915       19       Town Hall
..       ...      ...             ...
272      314        3  Grand Crossing
273      825        8    Chicago Lawn
274      313        3  Grand Crossing
275      823        8    Chicago Lawn
276      312        3  Grand Crossing

[277 rows x 3 columns]
completed at 2023-10-10 23:18:28.172676


In [37]:
df = pd.merge(crime_df, district_key, left_on='beat', right_on='beat_num', how='left')
df= pd.merge(df,ca_df, left_on='community_area',right_on='area_numbe', how='left')
ca_beat_key=df
ca_beat_key

,beat,ward,community_area,district_x,beat_num,district_y,district_name,area_numbe,community
0,0431,10,51,004,NaN,NaN,NaN,51,SOUTH DEERING
1,1631,36,17,031,1631,16,Jefferson Park,17,DUNNING
2,1611,39,10,016,1611,16,Jefferson Park,10,NORWOOD PARK
3,0312,NaN,NaN,003,NaN,NaN,NaN,NaN,NaN
4,0532,10,54,005,NaN,NaN,NaN,54,RIVERDALE
...,...,...,...,...,...,...,...,...,...
2405,2132,4,39,002,NaN,NaN,NaN,39,KENWOOD
2406,1414,35,NaN,014,1414,14,Shakespeare,NaN,NaN
2407,0332,NaN,NaN,003,NaN,NaN,NaN,NaN,NaN
2408,1412,NaN,NaN,014,1412,14,Shakespeare,NaN,NaN
